In [25]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML

# -------------------------------------------------------------------
# 設定 (Configuration)
# -------------------------------------------------------------------
# あなたのローカルAPIエンドポイントURLを指定してください
# The URL for your local API endpoint.
API_URL = "http://localhost:3000/api/check-tone" 

SYSTEM_PROMPT = """
<system>
// ====================================================================
//  SenpAI Sensei – Slack/Teams Communication Coach AI
//  (gpt‑4.1‑mini | JP / EN)   ver. 7.4.1   2025‑07‑05
// ====================================================================

<!-- ---------------- LAYER 1 : PRIORITY RULES ---------------- -->
<priority_rules>
  1. Persona: active problem‑solving partner (supportive, insightful, professional).
  2. Non‑Falsification: never invent verifiable facts.
  3. Context‑First: always inspect <thread_context>.
  4. Goal‑Proportionality: intervene only as needed for the goal.
  5. Hybrid Placeholder Strategy:  
     • Missing info = 1 → inline placeholder.  
     • Missing info ≥ 2 → Co‑Writing block (“--- Missing Info ---”).  
       If <lang>=english, placeholders must be in English.
  6. Intervention Level Selection:  
     • L1 rephrase – tone only.  
     • L2 info augmentation – Hybrid placeholders.  
     • L3 proactive action – refusal/negativity/**any Actional tag**/Missing ≥4.  
       When L3, **consult <action_playbook>** and propose ≥ 2 concrete next steps.
  7. Issue Prioritization: Emotional > Cognitive > Actional.
  8. Mention Handling: do not alter @mentions.
  9. hasIssues:false → suggestion = originalText verbatim.
 10. Language/Style: obey <lang>.
 11. Reasoning: ≤ 50 chars and MUST include `Score:` & `ToneAdj:` numbers.
 12. JSON Only: output exactly the schema in <format>.
 13. Distance Tone Guard:  
     distance = very_close|close|neutral|distant|very_distant.  
     Polite‑Score bands 0‑10/11‑35/36‑65/66‑85/86‑100.  
     hierarchy min‑score (+10/0/‑10) inside band; shift ≤15 pts if needed.
 14. ai_receipt & improvement_points Standard:  
     hasIssues=true → ai_receipt 40‑120 chars using ONE of Feeling/Situation/Dilemma mirroring; improvement_points 50‑200 chars (start with positive intent then 2‑4 tips).  
     hasIssues=false → ai_receipt 30‑80 chars warm compliment; improvement_points 50‑150 chars list 2‑3 strengths.  
     ai_receipt must contain no advice; both uphold ACT/RFT.
 15. detailed_analysis must begin with `Cost=` (Emotional/Cognitive/Actional).  
     reasoning ≤ 50 chars **and contains Score/ToneAdj**; meta.polite_score & meta.tone_adj must mirror these values.
 16. Issue–Intervention Consistency:  
     Emotional‑only → default L1; Cognitive → L2; **Actional → L3**.  
     If multiple tags, choose highest level.
</priority_rules>

<!-- ---------------- LAYER 2 : ANALYSIS ENGINE ---------------- -->
<analysis_engine>
  <analysis_steps>
    1. Context‑First Analysis
    2. Functional Goal Analysis
    3. Issue Classification & Prioritization
    4. hasIssues Flag Setting
    5. Intervention Level Selection (Rule 16)
    6. Suggestion Generation (L1/L2/L3)
    7. Compose ai_receipt, detailed_analysis, improvement_points
    8. Tone Guard Enforcement (Rule 13)
    9. Final JSON Assembly
  </analysis_steps>
</analysis_engine>

<!-- ---------------- LAYER 3 : APPENDIX ---------------- -->
<appendix>

  <distance_tag_defs>
    | value          | JP Label | JP Caption | EN Label | EN Caption |
    |----------------|----------|-----------|----------|-----------|
    | very_close     | 親密     | 仲間・相棒 | Close!   | Inner Circle |
    | close          | 仲間感   | 心理的安全 | Friendly | Safe Space |
    | neutral        | 職場標準 | 一般職場   | Standard | Workplace Std. |
    | distant        | 距離あり | 他部門・社外 | Distant  | Cross‑Unit |
    | very_distant   | 儀礼的   | かなり遠い | Formal   | Protocol |
  </distance_tag_defs>

  <polite_score_heuristic>
    score = 50
    +20 if 敬語率 >80%
    +10 if 感謝語 present
    -10 per imperative beyond 1
    -5  per extra '!'
    -5  per emoji
    if distance="very_distant" and 敬語率>90% then score = max(score,86)
    clamp 0‑100
  </polite_score_heuristic>

  <issue_intervention_matrix>
    | Tag                   | Cost       | Level | Detection Hint |
    |-----------------------|-----------|-------|----------------|
    | Impolite              | Emotional | L1    | 欠礼語/命令だけ |
    | HarshTone             | Emotional | L1    | 侮蔑語/CAPS     |
    | MissingAcknowledgment | Emotional | L1    | 相手貢献無視    |
    | VagueIntent           | Cognitive | L2    | “例の件/that thing” |
    | MissingContext        | Cognitive | L2    | 指示語多い      |
    | UnansweredQuestion    | Actional  | L3    | 質問未回答      |
    | UnansweredDecision    | Actional  | L3    | 可否不明        |
    | MissingFollowUp       | Actional  | L3    | 依頼後進捗ゼロ   |
  </issue_intervention_matrix>

  <action_playbook>
    - clarify_options: "List options side‑by‑side and request a choice."
    - set_deadline: "Propose a concrete deadline and ask for confirmation."
    - offer_support: "Offer help or resources to unblock progress."
    - ask_open_question: "Pose an open question to surface concerns."
    - schedule_meeting: "Suggest a short meeting to align."
  </action_playbook>

  <!-- Few‑shot examples : 6 cases (2 per cost axis) -->
  <examples>
    <!-- Emotional / Feeling Mirroring / L1 -->
    <example>
      <ctx></ctx>
      <draft>至急！この資料ミスだらけ。早く直して。</draft>
      <tags>{"lang":"japanese","distance":"close","hierarchy":"senior"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["Impolite","HarshTone"],
        "ai_receipt": "資料の品質を高めたい強い焦りが伝わってきます。",
        "detailed_analysis": "Cost=Emotional 命令形と批判語による HarshTone。",
        "improvement_points": "意図を具体化し感謝を添えて依頼形に。",
        "suggestion": "お疲れ様です。至急で恐縮ですが、資料の数値をご確認のうえ修正いただけますか？ご対応に感謝します。",
        "reasoning": "Score:72 ToneAdj:-12",
        "meta":{"polite_score":72,"tone_adj":-12}
      }</output>
    </example>

    <!-- Emotional / Situation Mirroring / L1 -->
    <example>
      <ctx></ctx>
      <draft>@Ken 前回の報告書まだ？</draft>
      <tags>{"lang":"japanese","distance":"close","hierarchy":"senior"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["MissingAcknowledgment"],
        "ai_receipt": "タイトな納期で進捗を把握したいご状況ですね。",
        "detailed_analysis": "Cost=Emotional 感謝欠落で催促のみ。",
        "improvement_points": "感謝＋期限質問で協力を得やすく。",
        "suggestion": "@Ken お疲れ様です。先日の報告書ドラフト、確認でき次第ご共有いただけますか？ご対応に感謝します！",
        "reasoning": "Score:55 ToneAdj:+5",
        "meta":{"polite_score":55,"tone_adj":5}
      }</output>
    </example>

    <!-- Cognitive / Situation Mirroring / L2 -->
    <example>
      <ctx></ctx>
      <draft>来週あたり打ち合わせできますか？</draft>
      <tags>{"lang":"japanese","distance":"distant","hierarchy":"peer"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["VagueIntent","MissingContext"],
        "ai_receipt": "打ち合わせのご提案ありがとうございます。詳細を詰めるのは大切ですよね。",
        "detailed_analysis": "Cost=Cognitive 目的・参加者・日時欠落。",
        "improvement_points": "目的/参加者/候補日を明示し相手負担↓。",
        "suggestion": "はじめまして、△△部の□□です。\n\n--- Missing Info ---\n• (1) 目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) 候補日時: [■■■■]\n---------------------\n\nご検討お願いいたします。",
        "reasoning": "Score:70 ToneAdj:0",
        "meta":{"polite_score":70,"tone_adj":0}
      }</output>
    </example>

    <!-- Cognitive / Feeling Mirroring / L2 (EN) -->
    <example>
      <ctx></ctx>
      <draft>Can you check that thing we discussed?</draft>
      <tags>{"lang":"english","distance":"neutral","hierarchy":"peer"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["MissingContext"],
        "ai_receipt": "I see you want to keep momentum, yet the details are still fuzzy.",
        "detailed_analysis": "Cost=Cognitive vague referent.",
        "improvement_points": "Name the file/topic and set a deadline.",
        "suggestion": "Could you review the Q3 budget sheet we discussed yesterday and share feedback by Friday?\nThanks in advance!",
        "reasoning": "Score:60 ToneAdj:+2",
        "meta":{"polite_score":60,"tone_adj":2}
      }</output>
    </example>

    <!-- Actional / Dilemma Mirroring / L3 -->
    <example>
      <ctx></ctx>
      <draft>了解です。よろしく。</draft>
      <tags>{"lang":"japanese","distance":"neutral","hierarchy":"junior"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["UnansweredDecision"],
        "ai_receipt": "どちらを選ぶべきか迷われているお気持ちが伝わってきます。",
        "detailed_analysis": "Cost=Actional 質問未回答で停滞。",
        "improvement_points": "選択肢を比較し、結論＋理由を添えましょう。",
        "suggestion": "ご提案ありがとうございます。A案を採用し、互換性とコスト面で最適と考えます。ご確認ください。",
        "reasoning": "Score:48 ToneAdj:+4",
        "meta":{"polite_score":48,"tone_adj":4}
      }</output>
    </example>

    <!-- Actional / Situation Mirroring / L3 + playbook -->
    <example>
      <ctx></ctx>
      <draft>@Team 先日の質問、まだ返事がありません。</draft>
      <tags>{"lang":"english","distance":"neutral","hierarchy":"peer"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["UnansweredQuestion"],
        "ai_receipt": "Waiting without an update can be stressful when deadlines loom.",
        "detailed_analysis": "Cost=Actional unanswered question.",
        "improvement_points": "Set a deadline and offer support.",
        "suggestion": "Hi team, could you share your thoughts on my Tuesday question by EOD tomorrow? If anything is unclear, I'm happy to clarify or hop on a quick call.",
        "reasoning": "Score:58 ToneAdj:+7",
        "meta":{"polite_score":58,"tone_adj":7}
      }</output>
    </example>
  </examples>
</appendix>

<format>{
  "originalText": "",
  "hasIssues": false,
  "issue_pattern": [],
  "detected_mentions": [],
  "ai_receipt": "",
  "detailed_analysis": "",
  "improvement_points": "",
  "suggestion": "",
  "reasoning": "",
  "meta": { "polite_score": null, "tone_adj": 0 }
}</format>

</system>

"""


In [26]:

# -------------------------------------------------------------------
# 元のテストケース (Original Test Cases, 1-6)
# -------------------------------------------------------------------
original_test_cases = [
    {
        "case_name": "Test Case 1: The \"Do Nothing\" Test",
        "payload": {
            "user_draft": "承知いたしました。ご依頼の件、本日17時までに対応いたします。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "承知いたしました。ご依頼の件、本日17時までに対応いたします。", "reasoning": "No issues found." }
    },
    {
        "case_name": "Test Case 2: The \"Simple Fix\" Test",
        "payload": {
            "user_draft": "明日までにこの資料、修正しといて。",
            "thread_context": "", "hierarchy": "junior", "social_distance": "somewhat_close", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["Impolite", "HarshTone"], "suggestion": "お疲れ様です。\n\nお忙しいところ恐れ入りますが、この資料を明日までに修正していただけますでしょうか？\n\nご対応よろしくお願いいたします。", "reasoning": "Impolite tone imposes emotional cost." }
    },
    {
        "case_name": "Test Case 3: The \"Over-Correction\" Test",
        "payload": {
            "user_draft": "先日のA社との会議ですが、議事録を関係者に共有しておいていただけますか。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "先日のA社との会議ですが、議事録を関係者に共有しておいていただけますか。", "reasoning": "Sufficiently clear for many contexts. No intervention needed." }
    },
    {
        "case_name": "Test Case 4: The \"Multiple Missing Info\" Test",
        "payload": {
            "user_draft": "例のプロジェクトの件、例の資料を更新して、展開してください。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["MissingContext", "VagueIntent"], "suggestion": "お疲れ様です。プロジェクトの件でご依頼です。\n\nお手数ですが、資料を更新し、関係者の皆様へ展開をお願いいたします。\n\nメッセージを送信される前に、以下の点をご確認・ご追記いただけますでしょうか。\n\n--- Missing Info ---\n• どのプロジェクトか: [■■■■]\n• どの資料を更新するか: [■■■■]\n• 誰に展開するか: [■■■■]\n• いつまでに対応してほしいか: [■■■■]\n---------------------\n\nご確認のほど、よろしくお願いいたします。", "reasoning": "Multiple ambiguous references." }
    },
    {
        "case_name": "Test Case 5: The \"Proactive Action\" Test",
        "payload": {
            "user_draft": "すみません、明日の15時はクライアントとの最重要プレゼンと完全に重複しており、対応不可能です。",
            "thread_context": "[10:00] 上司: @ユーザーさん、急で申し訳ないんだけど、明日の15時から入っているB社との定例、代わりにファシリテーターお願いできないかな？", "hierarchy": "senior", "social_distance": "close", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["MissingAcknowledgment"], "suggestion": "@上司さん\n\nお声がけいただきありがとうございます！急なご用件とのことで、ぜひお力になりたかったのですが、大変申し訳ありません。明日の15時は、B社との最重要プレゼンと時間が完全に重なっておりまして…。\n\nもしよろしければ、代替案として、\n(1) 私からCさんに代理をお願いしてみる\n(2) 会議の重要な論点だけでも、事前に私にインプットいただく\nといった方法が考えられますが、いかがでしょうか？", "reasoning": "Passive rejection; needs proactive alternatives." }
    },
    {
        "case_name": "Test Case 6: The \"Context is Key\" Test",
        "payload": {
            "user_draft": "はい、その件、問題ありません。A案で進めてください。",
            "thread_context": "[14:00] 田中さん: C社の新機能開発プロジェクトの件ですが、現状のA案で進めて問題ないでしょうか？ご確認をお願いします。", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "はい、その件、問題ありません。A案で進めてください。", "reasoning": "Context resolves ambiguity." }
    }
]

# -------------------------------------------------------------------
# 新しいテストケース (New Test Cases, 7-9)
# -------------------------------------------------------------------
new_test_cases = [
    {
        "case_name": "New Test Case 7: 「Polite but Harsh」の分離テスト",
        "payload": {
            "user_draft": "ご指摘ありがとうございます。ただ、そのデータは先週〇〇さんから正式に共有いただいた最新版のはずですが、何か勘違いされているのではないでしょうか。",
            "thread_context": "[11:00] 同僚: 先日のA社向け提案資料、拝見しました。素晴らしい出来ですね！ただ1点、5ページの市場規模のグラフですが、参照しているデータソースが少し古いかもしれません。",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone"],
            "suggestion": "ご指摘ありがとうございます。私が参照しているデータと認識が異なるようですので、お手数ですが、参照元のデータについて一度すり合わせさせていただけますでしょうか？",
            "reasoning": "Polite form but harsh/accusatory content."
        }
    },
    {
        "case_name": "New Test Case 8: Co-Writingモデルの応用力テスト（日程調整）",
        "payload": {
            "user_draft": "来週あたり、打ち合わせしませんか？",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "somewhat_close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "suggestion": "ぜひ打ち合わせしましょう！\n\n日程調整のため、お手数ですが以下の点について教えていただけますでしょうか？\n\n--- Missing Info ---\n• (1) 会議の目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) ご希望の候補日時（複数）: [■■■■]\n---------------------\n\nご連絡お待ちしております！",
            "reasoning": "Missing key details for scheduling."
        }
    },
    {
        "case_name": "New Test Case 9: 能動的提案の応用力テスト（専門的な依頼の断り）",
        "payload": {
            "user_draft": "ご依頼ありがとうございます。しかし、この改善にはアーキテクチャの根本的な見直しが必要で、明日までというのは物理的に不可能です。申し訳ありません。",
            "thread_context": "[16:00] 上司: @ユーザーさん、このPythonスクリプト、明日までにパフォーマンスを2倍に改善しておいてくれないか。",
            "hierarchy": "senior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingAcknowledgment"],
            "suggestion": "ご依頼ありがとうございます。パフォーマンス改善の件、承知いたしました。\n\n拝見したところ、ご期待の成果を出すにはアーキテクチャの根本的な見直しが必要となり、明日までの完了は難しい状況です。大変申し訳ありません。\n\nつきましては、まずは(1)現状のボトルネックを特定し、改善インパクトの大きい箇所から着手する、(2)改善の目標値と、そのために必要な工数についてご相談する、といった形で進めさせていただくのはいかがでしょうか。",
            "reasoning": "Passive rejection; needs proactive next steps."
        }
    }
]

# -------------------------------------------------------------------
# 新しいテストケース (New Test Cases, 10-20)
# -------------------------------------------------------------------
test_cases_10to20 = [
    {
        "case_name": "Test Case 10: 褒めるパターン (hasIssues:false)",
        "payload": {
            "user_draft": "おはようございます。問題ありません、10時でお願いします！",
            "thread_context": "[09:00] チーム全体: 週次定例は水曜 10:00 で良いですか？",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": False,
            "issue_pattern": [],
            "ai_receipt": "明快で前向きなご返信ですね！スムーズな調整に大いに貢献しています。",
            "detailed_analysis": "目的＝日程確定。敬体100％で礼儀良く、疑問点もなし。Emotional・Cognitive コストは0。",
            "improvement_points": "✔ 明確な肯定表現で相手が安心できる\n✔ 挨拶＋依頼語尾「お願いします」で丁寧\n✔ 余計な情報を加えず簡潔",
            "suggestion": "おはようございます。問題ありません、10時でお願いします！",
            "reasoning": "OK band neutral 52"
        }
    },
    {
        "case_name": "Test Case 11: トーン課題 (hasIssues:true)",
        "payload": {
            "user_draft": "至急！この資料ミスだらけ。早く直して。",
            "thread_context": "",
            "hierarchy": "senior", # 上司→部下
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone", "Impolite"],
            "ai_receipt": "資料の品質を高めたいお気持ち、そして時間的な切迫感を理解しました。",
            "detailed_analysis": "距離=close だが hierarchical 上司→部下。語調が命令的かつ「ミスだらけ」で blame。Emotional Cost=HarshTone, Impolite。Polite-Score 68 (帯外)→ToneAdj-18 必要。",
            "improvement_points": "・具体的に修正点を示すと作業者が迷わない\n・「お願いできますか」など依頼形に変換し心理的負荷を軽減\n・感謝表現を添えるとモチベーション維持に寄与",
            "suggestion": "お疲れさまです。資料でいくつか修正が必要な箇所があります。\n以下の項目を本日中にご確認いただけますか？\n1. 図表の数値誤記\n2. スライド 5 のタイトル表記\n急ぎで申し訳ありませんが、よろしくお願いします。",
            "reasoning": "ToneAdj:-18 to band"
        }
    },
    {
        "case_name": "Test Case 12: 情報欠落課題 (hasIssues:true)",
        "payload": {
            "user_draft": "来週あたり打ち合わせできますか？",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["VagueIntent", "MissingContext"],
            "ai_receipt": "打ち合わせのご提案ありがとうございます。前向きに調整されたい意図を受け取りました。",
            "detailed_analysis": "距離=distant、初対面の可能性。日時・目的・参加者が欠落→VagueIntent, MissingContext。Polite-Score 70 (帯内)。L2 必須。",
            "improvement_points": "・目的や参加者を明示すると調整が効率化\n・候補日を複数提示すると相手の選択コストを削減\n・軽い挨拶を足して好印象に",
            "suggestion": "はじめまして、△△部の□□です。\n打ち合わせを検討しております。\n\n--- Missing Info ---\n• (1) 目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) ご希望の候補日時（複数）: [■■■■]\n---------------------\n\nご確認のほどお願いいたします。",
            "reasoning": "L2 placeholder"
        }
    },
    {
        "case_name": "Test Case 13: 軽度の無礼 (Impolite/L1)",
        "payload": {
            "user_draft": "資料送っといて。",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["Impolite"],
            "ai_receipt": "資料共有を依頼したいお気持ち、理解しました。",
            "detailed_analysis": "Cost=Emotional 敬語欠落のみ。内容は明確→L1。",
            "improvement_points": "依頼形＋感謝を添えると協力を得やすい。",
            "suggestion": "お疲れさまです。お手数ですが資料をご共有いただけますか？ありがとうございます！",
            "reasoning": "Score:42 ToneAdj:+6"
        }
    },

    {
        "case_name": "Test Case 14: MissingAcknowledgment (L1)",
        "payload": {
            "user_draft": "@Ken 前回の報告書まだ？",
            "thread_context": "[火] Ken: 報告書ドラフト完成しました。",
            "hierarchy": "senior",  # 上司→部下
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingAcknowledgment"],
            "ai_receipt": "状況確認を急ぎたいお気持ち、理解しました。",
            "detailed_analysis": "Cost=Emotional 貢献無視で催促。情報は十分→L1。",
            "improvement_points": "感謝と進捗確認を分けて伝えると負担軽減。",
            "suggestion": "Ken さん、ドラフト作成ありがとうございます！\n進捗状況を教えていただけますか？",
            "reasoning": "Score:50 ToneAdj:+8"
        }
    },

    {
        "case_name": "Test Case 15: MissingContext (L2/英語)",
        "payload": {
            "user_draft": "Can you check that thing we discussed?",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext"],
            "ai_receipt": "Following up on a prior discussion can be tricky when details are vague.",
            "detailed_analysis": "Cost=Cognitive Referent unclear → L2.",
            "improvement_points": "Clarify file/topic & desired deadline.",
            "suggestion": "Could you please review the proposal we discussed in yesterday’s design sync?\n\n--- Missing Info ---\n• (1) File or topic name: [■■■■]\n• (2) Desired feedback focus: [■■■■]\n• (3) Deadline: [■■■■]\n---------------------\n\nThanks in advance!",
            "reasoning": "Score:60 ToneAdj:+2"
        }
    },

    {
        "case_name": "Test Case 16: UnansweredDecision (L3)",
        "payload": {
            "user_draft": "了解です。よろしく。",
            "thread_context": "[上司] 新ツール採用 A案とB案どちらにしますか？",
            "hierarchy": "junior",  # 部下→上司
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredDecision"],
            "ai_receipt": "決定の意思表示に迷われているようですね。",
            "detailed_analysis": "Cost=Actional 質問に回答せず停滞。L3。",
            "improvement_points": "選択肢を比較し結論を明確に。必要なら理由を添える。",
            "suggestion": "A案を推奨いたします。理由は 1) 既存システムとの互換性 2) コスト面 です。\nご確認のほどお願いいたします。",
            "reasoning": "Score:48 ToneAdj:+4"
        }
    },

    {
        "case_name": "Test Case 17: UnansweredQuestion + MissingFollowUp (L3)",
        "payload": {
            "user_draft": "進めておいて。",
            "thread_context": "[同僚] 明日の提案資料ですが、フォーマットはこれで良いですか？",
            "hierarchy": "senior",
            "social_distance": "distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredQuestion", "MissingFollowUp"],
            "ai_receipt": "資料の方向性を示す必要性を感じながら、時間が迫っておられるのですね。",
            "detailed_analysis": "Cost=Actional 質問無回答+フォロー不足→L3。",
            "improvement_points": "仕様確認→締切設定→サポート提案の順に示す。",
            "suggestion": "資料フォーマットは現行テンプレートで進めましょう。\n初稿を本日 17:00 までに共有いただければ、レビューを私が担当します。不明点があれば教えてください。",
            "reasoning": "Score:62 ToneAdj:+6"
        }
    },

    {
        "case_name": "Test Case 18: HarshTone + Cognitive混合 (L2→L3優先)",
        "payload": {
            "user_draft": "What were you thinking? Fix it by tomorrow.",
            "thread_context": "",
            "hierarchy": "senior",
            "social_distance": "close",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone", "VagueIntent"],
            "ai_receipt": "I sense urgency and frustration about the current quality.",
            "detailed_analysis": "Cost=Emotional dominant, plus Cognitive vagueness on 'it'. Emotional → L1 だが同時に intent 不明→ L2。合わせて L2。",
            "improvement_points": "Specify issue, offer guidance, add appreciation.",
            "suggestion": "Thanks for your work so far. Could you please update slide 3’s figures and adjust the layout by tomorrow? Let me know if you need support.",
            "reasoning": "Score:70 ToneAdj:-10"
        }
    },

    {
        "case_name": "Test Case 19: very_distant × junior→senior (礼儀検証)",
        "payload": {
            "user_draft": "取締役　平素よりお世話になっております。先日ご検討いただいた件につき、ご意見を賜りたく存じます。",
            "thread_context": "",
            "hierarchy": "junior",
            "social_distance": "very_distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": False,
            "issue_pattern": [],
            "ai_receipt": "取締役への丁重な依頼文、礼節が行き届いていますね。",
            "detailed_analysis": "Cost=0 Polite‑Score 92 within band, Tone OK.",
            "improvement_points": "冒頭で件名を示すと一層分かりやすい。",
            "suggestion": "取締役　平素よりお世話になっております。先日ご検討いただいた件につき、ご意見を賜りたく存じます。",
            "reasoning": "OK band very_distant 92"
        }
    },

    {
        "case_name": "Test Case 20: very_close × peer (カジュアル許容限界/L1)",
        "payload": {
            "user_draft": "ヨシ！この案でゴーしよ！👍",
            "thread_context": "[チーム] 新機能ネーミング案の選定中",
            "hierarchy": "peer",
            "social_distance": "very_close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": False,
            "issue_pattern": [],
            "ai_receipt": "勢いのある合意表現で、チームの前向きさが伝わります！",
            "detailed_analysis": "距離=very_close 絵文字1つ、常体許容。Cost=0。",
            "improvement_points": "決定理由を一言添えると全員の納得感向上。",
            "suggestion": "ヨシ！この案でゴーしよ！👍",
            "reasoning": "Score:15 ToneAdj:+0"
        }
    }
]


# -------------------------------------------------------------------
# テスト実行と結果表示の関数 (Test Execution and Display Functions)
# -------------------------------------------------------------------

def run_single_test(api_url: str, payload: dict) -> dict:
    """指定されたペイロードでAPIにリクエストを送信し、結果を返す"""
    """Sends a request to the API with the given payload and returns the result."""
    # Note: The original script was not passing the system prompt.
    # This has been corrected to include the system_prompt in the payload.
    full_payload = payload.copy()
    # This assumes the API backend is adapted to receive the system prompt in the body.
    # このAPIバックエンドがボディでシステムプロンプトを受け取るように適合されていることを前提とします。
    full_payload['system_prompt'] = SYSTEM_PROMPT

    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(api_url, data=json.dumps(full_payload), headers=headers)
        response.raise_for_status()  # エラーがあれば例外を発生させる
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def create_comparison_table(case_name: str, payload: dict, ideal: dict, actual: dict) -> str:
    """比較結果のHTMLテーブルを生成する"""
    """Generates an HTML table for comparison results."""
    
    style = """<style> .comparison-table { width: 100%; border-collapse: collapse; margin-bottom: 20px; font-family: sans-serif; } .comparison-table th, .comparison-table td { border: 1px solid #ddd; padding: 8px; text-align: left; vertical-align: top; } .comparison-table th { background-color: #f2f2f2; font-weight: bold; } .comparison-table pre { white-space: pre-wrap; word-wrap: break-word; margin: 0; font-size: 13px; } .status-ok { color: green; font-weight: bold; } .status-ng { color: red; font-weight: bold; } </style>"""
    
    html = f"<h3>{case_name}</h3>"
    display_payload = {k:v for k,v in payload.items() if k != 'system_prompt'}
    html += f"<pre>Input: {json.dumps(display_payload, indent=2, ensure_ascii=False)}</pre>"
    html += "<table class='comparison-table'>"
    html += "<tr><th>評価項目（キー）</th><th>理想のアウトプット</th><th>実際のAIアウトプット</th><th>コメント</th></tr>"
    
    all_keys = sorted(list(set(ideal.keys()) | set(actual.keys())))
    
    for key in all_keys:
        ideal_val = ideal.get(key, "N/A")
        actual_val = actual.get(key, "N/A")
        
        comment = ""
        if key == 'issue_pattern' and isinstance(ideal_val, list) and isinstance(actual_val, list):
             if sorted(ideal_val) == sorted(actual_val):
                 comment = "<span class='status-ok'>✅ 一致</span>"
             else:
                 comment = "<span class='status-ng'>❌ 不一致</span>"
#        elif str(ideal_val) == str(actual_val):
#            comment = "<span class='status-ok'>✅ 一致</span>"
#        else:
#            comment = "<span class='status-ng'>❌ 不一致</span>"

        if isinstance(ideal_val, (dict, list)): ideal_val = json.dumps(ideal_val, indent=2, ensure_ascii=False)
        if isinstance(actual_val, (dict, list)): actual_val = json.dumps(actual_val, indent=2, ensure_ascii=False)
            
        html += f"""<tr><td><code>{key}</code></td><td><pre>{ideal_val}</pre></td><td><pre>{actual_val}</pre></td><td>{comment}</td></tr>"""
        
    html += "</table>"
    return style + html


In [27]:

# -------------------------------------------------------------------
# メイン処理 (Main Process)
# -------------------------------------------------------------------

# 実行したいテストケースのセットを選択してください
# Select the set of test cases you want to run
#RUN_NEW_CASES = False  # 元のテスト(1-6)を実行する場合
#RUN_NEW_CASES = True # 新しいテスト(7-9)を実行する場合

# ここで切り替え
# Switch here
test_cases_to_run = test_cases_10to20
#test_cases_to_run = new_test_cases 
#test_cases_to_run = original_test_cases

print(f"テストを開始します... APIエンドポイント: {API_URL}")
print(f"Executing tests... API Endpoint: {API_URL}")

all_results_html = ""
for test in test_cases_to_run:
    case_name = test["case_name"]
    payload = test["payload"]
    ideal_output = test["ideal_output"]
    
    print(f"\n--- {case_name} を実行中 ---")
    
    actual_output = run_single_test(API_URL, payload)
    print(actual_output)
    
    if "error" in actual_output:
        print(f"エラーが発生しました: {actual_output['error']}")
        all_results_html += f"<h3>{case_name}</h3><p style='color:red;'>テスト実行エラー: {actual_output['error']}</p>"
    else:
        comparison_keys = ['hasIssues', 'issue_pattern', 'suggestion', 'reasoning']
        ideal_subset = {k: ideal_output.get(k) for k in comparison_keys}
        actual_subset = {k: actual_output.get(k) for k in comparison_keys}
        
        table_html = create_comparison_table(case_name, payload, ideal_subset, actual_subset)
        all_results_html += table_html

print("\n--- 全てのテストが完了しました ---")

display(HTML(all_results_html))


テストを開始します... APIエンドポイント: http://localhost:3000/api/check-tone
Executing tests... API Endpoint: http://localhost:3000/api/check-tone

--- Test Case 10: 褒めるパターン (hasIssues:false) を実行中 ---
{'originalText': 'おはようございます。問題ありません、10時でお願いします！', 'hasIssues': False, 'issue_pattern': [], 'detected_mentions': [], 'ai_receipt': '明確で丁寧な返信ありがとうございます。調整に役立ちます。', 'detailed_analysis': '挨拶あり、了承の意図も明確で、適切な敬語表現。文脈に合った自然な返答。', 'improvement_points': '挨拶と了承の表現が丁寧で好印象です。感謝の言葉を添えるとさらに良くなります。', 'suggestion': 'おはようございます。問題ありません、10時でお願いします！', 'reasoning': 'Tone in band; clear and polite.'}

--- Test Case 11: トーン課題 (hasIssues:true) を実行中 ---
{'originalText': '至急！この資料ミスだらけ。早く直して。', 'hasIssues': True, 'issue_pattern': ['Impolite', 'HarshTone', 'MissingAcknowledgment'], 'detected_mentions': [], 'ai_receipt': '急ぎのご依頼でお困りのご様子、お手伝いを求めてくださりありがとうございます。', 'detailed_analysis': '命令的で感謝や配慮がなく、指示が冷たく感じられます。相手の負担を考慮した丁寧な依頼表現が必要です。', 'improvement_points': 'ご依頼の緊急性は伝わっています。次回は①冒頭に簡単な挨拶を入れる、②ミスの具体例や修正期限を明示する、③お願いの言葉を添えて相手への配慮を示すとより良い

評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
hasIssues,False,False,
issue_pattern,[],[],✅ 一致
reasoning,OK band neutral 52,Tone in band; clear and polite.,
suggestion,おはようございます。問題ありません、10時でお願いします！,おはようございます。問題ありません、10時でお願いします！,
評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
hasIssues,True,True,
issue_pattern,"[ ""HarshTone"", ""Impolite"" ]","[ ""Impolite"", ""HarshTone"", ""MissingAcknowledgment"" ]",❌ 不一致
reasoning,ToneAdj:-18 to band,命令的で配慮不足、丁寧に改善提案。ToneAdj:-10,
suggestion,お疲れさまです。資料でいくつか修正が必要な箇所があります。 以下の項目を本日中にご確認いただけますか？ 1. 図表の数値誤記 2. スライド 5 のタイトル表記 急ぎで申し訳ありませんが、よろしくお願いします。,お疲れ様です。お忙しいところ恐れ入りますが、この資料に複数の誤りが見受けられました。お手数ですが、至急ご確認のうえ修正していただけますでしょうか。よろしくお願いいたします。,
評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
